### INF555 - Constraint-based Modeling and Algorithms for Decision Making 

### Ecole Polytechnique - Master of Artificial Intelligence & Advanced Visual Computing (MAI)

## TP (Theory in Practice) session 1: introduction to MiniZinc Jupyter Notebooks

This is a **Jupyter notebook** running in a **Docker image** and communicating with your browser on `port 8888` by default
- It runs **Python3**, so every cell will be executed with Python3, a dynamic object-oriented language (we'll come back to some Python basics later on)
- Shift-Enter executes a cell and goes to the next one
- A double clic on a markdown cell allows you to edit it

All TPs contain questions which you have to answer
* either directly in the notebook by **editing the markdown cell for textual answers**
* or by **creating a file** in Jupyter using the `new` button in file-explorer (upper-left `file open` menu)
* without forgetting to *download on your local machine* the notebook `TP.ipynb` with your files to *finally upload them on the Moodle* at the end of each session (late uploads will be accepted until midnight)

We will start all TPs by importing the module `inf555.py` which contains the necessary libraries to execute the constraint-based modelling language MiniZinc.

In [1]:
import inf555

You can open the corresponding file `inf555.py` 
* from the file-explorer (upper-left icon) 
* or in your browser by clicking [this link](inf555.py)
* or in your new [favorite editor](http://localhost:8080/?folder=/home/jovyan)


## Map coloring

A MiniZinc model is composed of
- variable declarations
- constraints
- a `solve` statement

Let us first have a look at a partial model (file `aust.mzn`) for coloring Australia such that two neighbor states have a different color.

We will have colorized output in the notebook through a shell command

In [2]:
!vimcat.sh aust.mzn

% Colouring Australia using nc colours
int: nc;

var 1..nc: wa;   var 1..nc: nt;  var 1..nc: sa;   var 1..nc: q;
var 1..nc: nsw;  var 1..nc: v;   var 1..nc: t;

constraint wa != nt;
constraint wa != sa;
constraint nt != sa;
constraint nt != q;
constraint sa != q;
constraint sa != nsw;
constraint sa != v;
constraint q != nsw;
constraint nsw != v;


### Question 1. What is missing to make a full MiniZinc model?

write your answer here by editing this markdown cell:


answer: the output statement


In [13]:
!vimcat.sh aust_solve.mzn

include "aust.mzn";

solve satisfy;

% output ["wa=", show(wa), "\t nt=", show(nt), 
%         "\t sa=", show(sa), "\n", "q=", show(q),  
%         "\t nsw=", show(nsw), "\t v=", show(v), "\n",
%         "t=", show(t),  "\n"];


In [14]:
!vimcat.sh aust.dzn

nc = 4;


Now the problem can be solved by running the following cell to get a solution (using the default constraint solver `gecode`)

In [15]:
print(inf555.minizinc('aust_solve.mzn', 'aust.dzn'))

Solution(wa=3, nt=2, sa=1, q=3, nsw=2, v=3, t=1, _checker='')


We can use Python
* to provide the data in Python instead of in a file
* to ask for statistics
* and by storing the solutions in a Python object

In [16]:
result = inf555.minizinc('aust_solve.mzn', data={'nc': 4})
print(result.solution)

Solution(wa=3, nt=2, sa=1, q=3, nsw=2, v=3, t=1, _checker='')


In [17]:
import pprint
pprint.pprint(result.statistics)

{'failures': 0,
 'flatIntConstraints': 9,
 'flatIntVars': 7,
 'flatTime': datetime.timedelta(microseconds=15327),
 'initTime': datetime.timedelta(microseconds=137),
 'method': 'satisfy',
 'nodes': 8,
 'paths': 0,
 'peakDepth': 7,
 'propagations': 9,
 'propagators': 9,
 'restarts': 0,
 'solutions': 1,
 'solveTime': datetime.timedelta(microseconds=72),
 'time': datetime.timedelta(microseconds=20000),
 'variables': 7}


In [6]:
print(result.status)

SATISFIED


We can also compute *all solutions* 
* by changing the default option
* and printing the array of solutions in python

In [9]:
result = inf555.minizinc('aust_solve.mzn', data={'nc': 4}, all_solutions=True)
print(result.statistics['solveTime'])
print(result.status)

0:00:00.006239
ALL_SOLUTIONS


In [10]:
result.solution[0:10]

[Solution(wa=3, nt=2, sa=1, q=3, nsw=2, v=3, t=1, _checker=''),
 Solution(wa=4, nt=2, sa=1, q=3, nsw=2, v=3, t=1, _checker=''),
 Solution(wa=3, nt=2, sa=1, q=4, nsw=2, v=3, t=1, _checker=''),
 Solution(wa=4, nt=2, sa=1, q=4, nsw=2, v=3, t=1, _checker=''),
 Solution(wa=3, nt=2, sa=1, q=3, nsw=2, v=4, t=1, _checker=''),
 Solution(wa=4, nt=2, sa=1, q=3, nsw=2, v=4, t=1, _checker=''),
 Solution(wa=3, nt=2, sa=1, q=4, nsw=2, v=4, t=1, _checker=''),
 Solution(wa=4, nt=2, sa=1, q=4, nsw=2, v=4, t=1, _checker=''),
 Solution(wa=3, nt=2, sa=1, q=3, nsw=2, v=3, t=2, _checker=''),
 Solution(wa=4, nt=2, sa=1, q=3, nsw=2, v=3, t=2, _checker='')]

To come back to MiniZinc, there are three different `solve` statements:

- `solve satisfy`
- `solve minimize`
- `solve maximize`

In [11]:
!vimcat.sh aust_opt.mzn

include "aust.mzn";

var int: max;

constraint max >= max([wa, nt, sa, q, nsw, v, t]);

solve minimize max;


In [12]:
inf555.minizinc('aust_opt.mzn', data={'nc': 4}).solution

Solution(objective=3, max=3, wa=3, nt=2, sa=1, q=3, nsw=2, v=3, t=1, _checker='')

MiniZinc has a quite rich syntax

with enumerated types, complex and reified constraints, arrays, sets and loops.

In [13]:
!vimcat.sh aust_enum.mzn

enum Color;
var Color: wa; 
var Color: nt; 
var Color: sa;
var Color: q;
var Color: nsw;
var Color: v;
var Color: t;
constraint wa != nt /\ wa != sa /\ nt != sa /\ nt != q /\ sa != q;
constraint sa != nsw /\ sa != v /\ q != nsw /\ nsw != v; 
solve satisfy;


In [14]:
import enum
Color = enum.Enum('Color', ['red', 'green', 'blue'])
print(inf555.minizinc('aust_enum.mzn', data={'Color': Color}))

Solution(wa=<Color.blue: 3>, nt=<Color.green: 2>, sa=<Color.red: 1>, q=<Color.blue: 3>, nsw=<Color.green: 2>, v=<Color.blue: 3>, t=<Color.red: 1>, _checker='')


In [15]:
!vimcat.sh magic_series.mzn

int: n;
array[0..n-1] of var 0..n: s;

constraint forall(i in 0..n-1) (
   s[i] = (sum(j in 0..n-1) (bool2int(s[j]=i))));

solve satisfy;   


In [16]:
result = inf555.minizinc('magic_series.mzn', data={'n': 8})
result.solution

Solution(s=[4, 2, 1, 0, 1, 0, 0, 0], _checker='')

You should avoid writing Python code directly in the notebooks (even though it is possible).

Please write python code in the [editor](http://localhost:8080/) and don't forget to download it to your local machine in order to upload it with your other answers on the Moodle.

---

## Wedding 

Here is another example, we will use the `output` command from MiniZinc to get an easier to read display of the result.

In [18]:
!vimcat.sh wedding.mzn

enum Guests = {bride, groom, bestman, brdmaid, bob, carol,
               ted, alice, ron, rona, ed, clara};
set of int: Seats = 1..12;
set of int: Hatreds = 1..5;
array[Hatreds] of Guests: h1 = [groom, carol, ed, bride, ted];
array[Hatreds] of Guests: h2 = [clara, bestman, ted, alice, ron];
set of Guests: Males = {groom, bestman, bob, ted, ron, ed};
/* very different from
 * var {groom, bestman, bob, ted, ron, ed}: Males
 * which would define the domain of variable Males */
set of Guests: Females = {bride, brdmaid, carol, alice, rona, clara};

array[Guests] of var Seats: pos;       % seat of guest
array[Hatreds] of var Seats: p1;       % seat of guest 1 in hatred
array[Hatreds] of var Seats: p2;       % seat of guest 2 in hatred
array[Hatreds] of var 0..1: sameside;  % seats of hatred on same side
array[Hatreds] of var Seats: cost;     % penalty of hatred

include "alldifferent.mzn";
constraint alldifferent(pos);
constraint forall(g in Males) (pos[g] mod 2 == 1);
constraint forall([po

It is possible to select other solvers, here we switch from the default [Gecode](http://gecode.org) to the [Chuffed](https://github.com/chuffed/chuffed) solver

In [19]:
result = inf555.minizinc('wedding.mzn', solver=inf555.chuffed)
print(result.statistics['solveTime'])
print(result)
result.solution

0:00:00.006000
ron	clara	bob	rona	bestman	alice
ted	brdmaid	groom	bride	ed	carol	



Solution(objective=22, pos=[10, 9, 5, 8, 3, 12, 7, 6, 1, 4, 11, 2], p1=[9, 12, 11, 10, 7], p2=[2, 5, 7, 6, 1], sameside=[0, 0, 1, 0, 0], cost=[3, 5, 4, 4, 6], _output_item='ron\tclara\tbob\trona\tbestman\talice\nted\tbrdmaid\tgroom\tbride\ted\tcarol\t\n', _checker='')

### Question 2. What does the above minizinc program do?

write your answer here

users use minizinc model to describe a mathematical problem 
then minizinc program translate the model into suitable forms for some solvers 
in the end the solver give one or some answers

## Optimization

Let us go back to our Australia coloring problem.

Now, let's suppose that we want our minimization problem to be parametric, using the same techniques as in the wedding model above.

We have the following data file:

In [18]:
!vimcat.sh aust_param.dzn

Vertices = {wa, nt, sa, q, nsw, v};
NbEdges = 1..9;
Edges1 = [wa, wa, nt, nt, sa, sa, sa, q, nsw];
Edges2 = [nt, sa, sa, q, q, nsw, v, nsw, v];


### Question 3. Write the [aust_param.mzn](aust_param.mzn) model that minimizes `nc` as before
such that the following call gives us the expected result***

In [29]:
inf555.minizinc('aust_param.mzn', 'aust_param.dzn')

Result(status=<Status.OPTIMAL_SOLUTION: 7>, solution=Solution(objective=3, max=3, color=[3, 2, 1, 3, 2, 3], _checker=''), statistics={'paths': 0, 'flatIntVars': 8, 'flatIntConstraints': 11, 'method': 'minimize', 'flatTime': datetime.timedelta(microseconds=18168), 'time': datetime.timedelta(microseconds=20000), 'initTime': datetime.timedelta(microseconds=1097), 'solveTime': datetime.timedelta(microseconds=491), 'solutions': 1, 'variables': 8, 'propagators': 11, 'propagations': 40, 'nodes': 9, 'failures': 3, 'restarts': 0, 'peakDepth': 7})

## Loan

In [30]:
!vimcat.sh loan.mzn

% variables
var float: R;        % quarterly repayment
var float: P;        % principal initially borrowed
var 0.0 .. 10.0: I;  % interest rate

% intermediate variables
var float: B1; % balance after one quarter
var float: B2; % balance after two quarters
var float: B3; % balance after three quarters
var float: B4; % balance owing at end

constraint B1 = P * (1.0 + I) - R;
constraint B2 = B1 * (1.0 + I) - R;
constraint B3 = B2 * (1.0 + I) - R; 
constraint B4 = B3 * (1.0 + I) - R;

solve satisfy;

output [
   "Borrowing ", show_float(0, 2, P), " at ", show(I*100.0), 
   "% interest, and repaying ", show_float(0, 2, R), 
   "\nper quarter for 1 year leaves ", show_float(0, 2, B4), " owing\n"
];


In [31]:
result = inf555.minizinc('loan.mzn', data={'P': 1000.0, 'R': 260.0, 'I': 0.04}, solver=inf555.cbc)
print(result)
result.solution

Borrowing 1000.00 at 4.0% interest, and repaying 260.00
per quarter for 1 year leaves 65.78 owing



Solution(R=260.0, P=1000.0, I=0.04, B1=780.0, B2=551.2, B3=313.248, B4=65.77792000000005, _output_item='Borrowing 1000.00 at 4.0% interest, and repaying 260.00\nper quarter for 1 year leaves 65.78 owing\n', _checker='')

In [32]:
pprint.pprint(result.statistics)

{'flatFloatConstraints': 3,
 'flatFloatVars': 4,
 'flatTime': datetime.timedelta(microseconds=24140),
 'method': 'satisfy',
 'nodes': 0,
 'objective': 0,
 'objectiveBound': 0,
 'openNodes': -1,
 'paths': 0,
 'solveTime': datetime.timedelta(microseconds=4900),
 'time': datetime.timedelta(microseconds=40000)}


### Question 4. Write the corresponding loan.dzn file
* before running the next cell to answer the question:
* "How much do I need to pay per quarter if I want to borrow $1500 at 5% interest rate and owe nothing after 1 year?"



In [36]:
inf555.minizinc('loan.mzn', 'loan.dzn', solver=inf555.cbc)

Result(status=<Status.SATISFIED: 5>, solution=Solution(R=423.0177489051942, P=1500.0, I=0.05, B1=1151.982251094806, B2=786.563614744352, B3=402.8740465763754, B4=0.0, _output_item='Borrowing 1500.00 at 5.0% interest, and repaying 423.02\nper quarter for 1 year leaves 0.00 owing\n', _checker=''), statistics={'paths': 0, 'flatFloatVars': 4, 'flatFloatConstraints': 4, 'method': 'satisfy', 'flatTime': datetime.timedelta(microseconds=20706), 'objective': 0, 'objectiveBound': 0, 'nodes': 0, 'solveTime': datetime.timedelta(microseconds=400), 'time': datetime.timedelta(microseconds=20000), 'openNodes': -1})

To make the number of quarters a parameter, we will use the fact that MiniZinc allows us to define predicates, i.e. functions returning a boolean expression.

In [42]:
!vimcat.sh loan_predicate.mzn

% parameter
int: D;              % duration in quarters
% variables
var float: R;        % quarterly repayment
var float: P;        % principal initially borrowed
var 0.0 .. 10.0: I;  % interest rate
var float: B;        % balance owing at end

% remember that you can use
% if/then/else/endif
% it is also possible to declare local variables like
% let { var float: b; } in
predicate cumulative_interest(
   var float: balance, var float: principal, var float: interest,
   var float: repayment, int: nbquarters) = 
   if (balance ==0)
   then 
      cumulative_interest(
         (principal * (1 + interest) - repayment ), principal,
         interest, repayment, (nbquarters-1)
         )
   else
      cumulative_interest(
         (balance * (1 + interest) - repayment ), principal,
         interest, repayment, (nbquarters-1)
         )

constraint cumulative_interest(B, P, I, R, D);

solve satisfy;

output [
   "Borrowing ", show_float(0, 2, P), " at ", show(I*100.0), 
   "% interest, and 

### Question 5. Modify the [loan_predicate.mzn](loan_predicate.mzn) file in order to recursively define our cumulative interest

### Question 6.  Write the query to know 
* how much one should pay quarterly
* in order to have a balance of 0.0
* after 3 years
* of borrowing $1000
* at a 4% interest rate


In [49]:
inf555.minizinc('loan_predicate.mzn', data={'D':12,'B':0.0,'P':1000.0,'I':0.04}, solver=inf555.cbc)

MiniZincError: MiniZinc stopped with a non-zero exit code, but did not output an error message. 

I meet unexpeted error " MiniZinc stopped with a non-zero exit code, but did not output an error message. " 

and Warining "# We use the `send` method directly, because coroutines # don't have `__iter__` and `__next__` methods."

*Do not forget to download*
* `TP.ipynb`
* `aust_param.mzn`
* `loan.dzn`
* and the modified `loan_predicate.mzn` 

files to your machine (or work/ directory) 

and to *upload them* to the **Assignment** module of TP1 on the Moodle